# Introduction to the data

In [1]:
import sqlite3
conn = sqlite3.connect('nominations.db')
schema_query = "pragma table_info(nominations);"
schema = conn.execute(schema_query).fetchall()
first_ten_query = '''SELECT * FROM nominations LIMIT 10'''
first_ten = conn.execute(first_ten_query).fetchall()

for item in schema:
    print(item)
    
for item in first_ten:
    print(item)

(0, u'Year', u'INTEGER', 0, None, 0)
(1, u'Category', u'TEXT', 0, None, 0)
(2, u'Nominee', u'TEXT', 0, None, 0)
(3, u'Won', u'INTEGER', 0, None, 0)
(4, u'Movie', u'TEXT', 0, None, 0)
(5, u'Character', u'TEXT', 0, None, 0)
(2010, u'Actor -- Leading Role', u'Javier Bardem', 0, u'Biutiful', u'Uxbal')
(2010, u'Actor -- Leading Role', u'Jeff Bridges', 0, u'True Grit', u'Rooster Cogburn')
(2010, u'Actor -- Leading Role', u'Jesse Eisenberg', 0, u'The Social Network', u'Mark Zuckerberg')
(2010, u'Actor -- Leading Role', u'Colin Firth', 1, u"The King's Speech", u'King George VI')
(2010, u'Actor -- Leading Role', u'James Franco', 0, u'127 Hours', u'Aron Ralston')
(2010, u'Actor -- Supporting Role', u'Christian Bale', 1, u'The Fighter', u'Dicky Eklund')
(2010, u'Actor -- Supporting Role', u'John Hawkes', 0, u"Winter's Bone", u'Teardrop')
(2010, u'Actor -- Supporting Role', u'Jeremy Renner', 0, u'The Town', u'James Coughlin')
(2010, u'Actor -- Supporting Role', u'Mark Ruffalo', 0, u'The Kids Are A

# Creating The Ceremonies Table

In [21]:
ceremony_table_query = "CREATE TABLE ceremonies(id INTEGER PRIMARY KEY,Year INTEGER, Host TEXT);"
ceremonies = conn.execute(ceremony_table_query)

In [3]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal")]

insert_query = "INSERT INTO ceremonies(Year, Host) VALUES(?,?);"
insert_rows2_ceremonies = conn.executemany(insert_query, years_hosts)

In [4]:
query = "SELECT * FROM ceremonies LIMIT 10"
conn.execute(query).fetchall()

[(1, 2010, u'Steve Martin'),
 (2, 2009, u'Hugh Jackman'),
 (3, 2008, u'Jon Stewart'),
 (4, 2007, u'Ellen DeGeneres'),
 (5, 2006, u'Jon Stewart'),
 (6, 2005, u'Chris Rock'),
 (7, 2004, u'Billy Crystal'),
 (8, 2003, u'Steve Martin'),
 (9, 2002, u'Whoopi Goldberg'),
 (10, 2001, u'Steve Martin')]

In [5]:
pragma_q = "PRAGMA table_info(ceremonies);"
conn.execute(pragma_q).fetchall()

[(0, u'id', u'INTEGER', 0, None, 1),
 (1, u'Year', u'INTEGER', 0, None, 0),
 (2, u'Host', u'TEXT', 0, None, 0)]

# Foreign Key Constraints

In [6]:
fkq = "PRAGMA foreign_keys = ON;"
conn.execute(fkq)

# Setting Up One-To-Many

In [7]:
nominations_two_table_query = '''CREATE TABLE nominations_two
(id INTEGER PRIMARY KEY,
category text, nominee TEXT,
movie TEXT,
character TEXT,
won INTEGER,
ceremony_id INTEGER,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id));'''

nominations_two = conn.execute(nominations_two_table_query)

In [8]:
nominations_two_values = '''SELECT nominations.category, nominations.nominee,
nominations.movie, nominations.character, nominations.won,
ceremonies.id 
FROM nominations
INNER JOIN ceremonies 
ON nominations.year == ceremonies.year;'''

joined_nominations = conn.execute(nominations_two_values)

In [9]:
insert_nominations_two_q = '''INSERT INTO nominations_two(category,nominee, movie,
character, won, ceremony_id) VALUES(?,?,?,?,?,?);'''
insert_nominations_two = conn.executemany(insert_nominations_two_q, joined_nominations )

In [10]:
que = "SELECT * FROM nominations_two LIMIT 5;"
conn.execute(que).fetchall()

[(1, u'Actor -- Leading Role', u'Javier Bardem', u'Biutiful', u'Uxbal', 0, 1),
 (2, u'Actor -- Leading Role', u'Javier Bardem', u'Biutiful', u'Uxbal', 0, 12),
 (3,
  u'Actor -- Leading Role',
  u'Jeff Bridges',
  u'True Grit',
  u'Rooster Cogburn',
  0,
  1),
 (4,
  u'Actor -- Leading Role',
  u'Jeff Bridges',
  u'True Grit',
  u'Rooster Cogburn',
  0,
  12),
 (5,
  u'Actor -- Leading Role',
  u'Jesse Eisenberg',
  u'The Social Network',
  u'Mark Zuckerberg',
  0,
  1)]

# Deleting And Renaming Tables

In [11]:
del_query = "DROP TABLE nominations;"
conn.execute(del_query)

In [12]:
rename_query = "ALTER TABLE nominations_two RENAME TO nominations;"
conn.execute(rename_query)

In [22]:
# checks to see del and renames
#print(conn.execute("SELECT * FROM nominations_two LIMIT 5;").fetchall())

#print(conn.execute("SELECT * FROM nominations LIMIT 5;").fetchall())

# Creating A Join Table

In [14]:
movies_tab_q = "CREATE TABLE movies(id INTEGER PRIMARY KEY, movie text)"
create_movies = conn.execute(movies_tab_q)

In [15]:
actors_tab_q = "CREATE TABLE actors(id INTEGER PRIMARY KEY, actor text)"
create_actors = conn.execute(actors_tab_q)

In [17]:
create_movies_actors = '''CREATE TABLE movies_actors (id INTEGER PRIMARY KEY, movie_id INTEGER REFERENCES movies(id),
actor_id INTEGER REFERENCES actors(id));'''
conn.execute(create_movies_actors)

In [18]:
# checks schemas 
pragma_q1 = "PRAGMA table_info(movies);"
conn.execute(pragma_q).fetchall()

[(0, u'id', u'INTEGER', 0, None, 1),
 (1, u'Year', u'INTEGER', 0, None, 0),
 (2, u'Host', u'TEXT', 0, None, 0)]

# Populating the movies and actors tables

In [19]:
insert_movies = "insert into movies (movie) select distinct movie from nominations;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn.execute(insert_movies)
conn.execute(insert_actors)

print(conn.execute("select * from movies limit 5;").fetchall())
print(conn.execute("select * from actors limit 5;").fetchall())

[(1, u'Biutiful'), (2, u'True Grit'), (3, u'The Social Network'), (4, u"The King's Speech"), (5, u'127 Hours')]
[(1, u'Javier Bardem'), (2, u'Jeff Bridges'), (3, u'Jesse Eisenberg'), (4, u'Colin Firth'), (5, u'James Franco')]


# Populating a join table

In [20]:
pairs_query = "select movie,nominee from nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()

join_table_insert = '''insert into movies_actors (movie_id, actor_id) values 
((select id from movies where movie == ?),(select id from actors where actor == ?));'''
conn.executemany(join_table_insert,movie_actor_pairs)

print(conn.execute("select * from movies_actors limit 5;").fetchall())

[(1, 1, 1), (2, 1, 1), (3, 2, 2), (4, 2, 2), (5, 3, 3)]
